In [1]:
"""purpose - create a persistent DuckDB database and store data from a couple datasets (i.e., as if they had been imported from CSV flatfiles) in it, each as their own table"""
from contextlib import contextmanager
import pathlib as pl
import sys
import warnings

import appdirs
import duckdb
import pandas as pd
from pydataset import data
from IPython.display import Markdown


# HACK the path for allow this notebook to import server's modules
THIS_DIR = globals()['_dh'][0]
REPO_ROOT = THIS_DIR.parent
SRC_DIR = (REPO_ROOT/'src').resolve()
#print(f"{THIS_DIR=}\n{REPO_ROOT=}\n{SRC_DIR=}")
sys.path.append(str(SRC_DIR))

from backend.sql.base import DUCKDB #<-- this import statement will implicitly create the DB if it doesn't exist
from backend.sql.duck import DuckDBMonitorMiddleware


data_dir = pl.Path(appdirs.site_data_dir("DemoApp", "JB"))
data_dir.mkdir(parents=True, exist_ok=True)

@contextmanager
def ignore_warnings():
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        yield

In [2]:
df_iris = data('iris')
df_titanic = data('titanic')
df_inequality = data('incomeInequality')

with duckdb.connect(DUCKDB.PATH) as conn:
    # create or update tables
    conn.sql("""
        CREATE OR REPLACE TABLE iris AS SELECT * FROM df_iris;
        CREATE OR REPLACE TABLE titanic AS SELECT * FROM df_titanic; 
        CREATE OR REPLACE TABLE inequality AS SELECT * FROM df_inequality;
    """)

    # WAL cleanup
    conn.sql("CHECKPOINT;")

In [8]:
df1 = DuckDBMonitorMiddleware.get_dataframe("SELECT * FROM iris;")
display(df1)
#df1['Species'] = df1['Species'].astype('category')
display(f"{isinstance(df1['Species'].dtype, pd.CategoricalDtype)=}")

with ignore_warnings(), duckdb.connect(DUCKDB.PATH) as conn:
    df2 = pd.read_sql("SELECT * FROM administrative.usage_tracking;", conn)
display(df2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


"isinstance(df1['Species'].dtype, pd.CategoricalDtype)=False"

,table_name,hits,last_hit
0,inequality,0,2025-08-18 10:26:45.430
1,iris,596,2025-08-25 10:51:44.789
2,titanic,0,2025-08-18 10:26:45.430


True